# 금보원 데이터챌린지_계획서 제출 코드

## 1. 데이터셋 및 라이브러리 불러오기

In [2]:
import pandas as pd

In [7]:
samsung = pd.read_csv(r"E:\금보원_데이터챌린지대회\삼성카드_data.csv",encoding='cp949')
insurance = pd.read_csv(r"E:\금보원_데이터챌린지대회\보험개발원_data.csv",encoding='cp949')

### 데이터셋 concat

In [10]:
df= pd.concat([samsung, insurance], axis=1)

In [174]:
df = df.fillna(0)

In [179]:
df["보험료"]=df["보험료"].apply(lambda x: x.strip().replace(",","")).astype(int)
df["지급금액"]=df["지급금액"].apply(lambda x: x.strip().replace(",","")).astype(int)

In [180]:
df["자동차_사고율"]= df["지급금액"]/df["보험료"]
df["장기_사고율"]=df["지급금액"]/df["보험료"]
df["생명_사고율"]=df["사고건수"]/df["계약건수"]

사고율 feature 추가

## 2. 데이터 분석 

## 3. feature 생성

- 평균 매출 금액 = 매출 금액 합 / 매출 금액 건수

1) 대분류별 데이터셋 생성

In [182]:
big_list = ["요식","자동차","교육","건강","여가","생활","유흥","쇼핑","여행"]

In [183]:
alpha = ["A","B","C","D","E","F","G"]

In [184]:
data = df[['기준년도', 'AGE', 'GENDER', 'JOB', 'INCOME', 'HOM_MGPO', 'HOM_SGG',
       'OFFI_MGPO', 'OFFI_SGG', 'ONEPER_GD_C', 'EST_LFSTG', 'UMRD_SCORE',
       'WEDD_EXP_SCORE', 'CHDB_EXP_SCORE', 'PSCH_OCH_SCORE', 'ESTUD_OCH_SCORE',
       'MHSTUD_OCH_SCORE', 'CSTUD_OCH_SCORE', 'OCH_WEDD_EXP_SCORE',
       'HOUSEWF_SCORE', 'ONL_PRSN', 'ERADP_TNDC', 'PRMM_CSM_TNDC', 'MDLV_PRSN',
       'PET_PRSN', 'ODS_ACTI_INRT_TNDC', 'FRN_TRV_INRT_TNDC',
       'INTERI_SLNG_TNDC', 'INTERI_REMDL_TNDC', 'HL_MNGT_INRT_TNDC']].copy()

In [185]:
data.loc[:,'총사용건수_합'] = (df['총사용건수_A'].values+df['총사용건수_B'].values+df['총사용건수_C'].values+df['총사용건수_D'].values+df['총사용건수_E'].values)
#data["총사용금액_합"] = df['총사용금액_A']+df['총사용금액_B']+df['총사용금액_C']+df['총사용금액_D']+df['총사용금액_E']

In [186]:
data.loc[:,'총사용금액_합'] = (df['총사용금액_A'].values+df['총사용금액_B'].values+df['총사용금액_C'].values+df['총사용금액_D'].values+df['총사용금액_E'].values)

In [187]:
data.loc[:,'총_평균금액_합'] = data['총사용금액_합']/data['총사용건수_합']

In [188]:
for i in big_list:
    data.loc[:,i+'_총사용건수_합']=0
    for j in alpha:
        data.loc[:,i+'_총사용건수_합'] += df["매출건수합계_"+i+"_"+j].values

In [189]:
for i in big_list:
    data.loc[:,i+'_총사용금액_합']=0
    for j in alpha:
        data.loc[:,i+'_총사용금액_합'] += df["매출금액합계_"+i+"_"+j].values

In [190]:
for i in big_list:
    data.loc[:,i+'_평균사용금액']= data[i+'_총사용금액_합']/data[i+'_총사용건수_합']

## 2. 등급 부여하기
- 등급은 값이 높을 수록 평균 사용금액이 높은 것을 의미한다.

In [191]:
def grade(bin_number,series):
    min_ = min(series.values)
    max_ = max(series.values)
    bin_size = (max_-min_)/bin_number
    grade_list = series.values
    # 등급 생성
    grade_list = []
    data = min_
    for i in range(bin_number+1):
        grade_list.append(data)
        data+= bin_size
    return grade_list

In [192]:
def grade_apply(x,columns_list):
    for i in range(len(columns_list)-1) :
        if x<=columns_list[i+1]:
            return i+1
    return len(columns_list)-1

In [88]:
# grade_list 전역변수로 지정 

In [193]:
columns_grade_list = ["요식_평균사용금액", '자동차_평균사용금액', '교육_평균사용금액', '건강_평균사용금액', '여가_평균사용금액',
       '생활_평균사용금액', '유흥_평균사용금액', '쇼핑_평균사용금액','UMRD_SCORE',
       'WEDD_EXP_SCORE', 'CHDB_EXP_SCORE', 'PSCH_OCH_SCORE', 'ESTUD_OCH_SCORE',
       'MHSTUD_OCH_SCORE', 'CSTUD_OCH_SCORE', 'OCH_WEDD_EXP_SCORE',
       'HOUSEWF_SCORE', 'ONL_PRSN', 'ERADP_TNDC', 'PRMM_CSM_TNDC', 'MDLV_PRSN',
       'PET_PRSN', 'ODS_ACTI_INRT_TNDC', 'FRN_TRV_INRT_TNDC',
       'INTERI_SLNG_TNDC', 'INTERI_REMDL_TNDC', 'HL_MNGT_INRT_TNDC']
 

In [194]:
for c in columns_grade_list:
    grade_list = grade(10,data[c]) # 분류별 범위 리스트 생성
    data.loc[:,c+"등급"]=data[c].apply(grade_apply,args=(grade_list,)) # apply 작성

# 3. 대분류 그래프 생성

## 1)사고율 별 그룹 생성

data drop

In [204]:
data = data[['AGE', 'GENDER', 'JOB', 'INCOME','요식_평균사용금액등급',
       '자동차_평균사용금액등급', '교육_평균사용금액등급', '건강_평균사용금액등급', '여가_평균사용금액등급',
       '생활_평균사용금액등급', '유흥_평균사용금액등급', '쇼핑_평균사용금액등급', 'UMRD_SCORE등급',
       'WEDD_EXP_SCORE등급', 'CHDB_EXP_SCORE등급', 'PSCH_OCH_SCORE등급',
       'ESTUD_OCH_SCORE등급', 'MHSTUD_OCH_SCORE등급', 'CSTUD_OCH_SCORE등급',
       'OCH_WEDD_EXP_SCORE등급', 'HOUSEWF_SCORE등급', 'ONL_PRSN등급', 'ERADP_TNDC등급',
       'PRMM_CSM_TNDC등급', 'MDLV_PRSN등급', 'PET_PRSN등급', 'ODS_ACTI_INRT_TNDC등급',
       'FRN_TRV_INRT_TNDC등급', 'INTERI_SLNG_TNDC등급', 'INTERI_REMDL_TNDC등급',
       'HL_MNGT_INRT_TNDC등급']].copy()

In [215]:
columns = data.columns

In [206]:
data = data.reset_index()

## 2) 그래프 생성

In [207]:
biggroup_dic = {}

In [221]:
biggroup_dic = {}
for idx,row in data.iterrows():
    for r in range(1,len(row)):
        value_ = columns[r]+"_"+str(row[r])
        if value_ in biggroup_dic:
            biggroup_dic[value_].append(idx)
        else:
            biggroup_dic[value_]= [idx]

## 3) 조건에 따라 불필요한 feature 필터링